<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/simulate_H2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 25.2 MB/s eta 0:00:00


In [3]:
import pennylane as qml
from pennylane import qchem
import numpy as np


# Define the molecular structure of H2
symbols = ["H", "H"]
coordinates = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.74]])  # Bond length of 0.74 angstroms

# Define the electronic Hamiltonian
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)

# Define the variational ansatz
def ansatz(params, wires):
    qml.BasisState(np.array([1, 1] + [0] * (qubits - 2)), wires=wires)
    qml.DoubleExcitation(params[0], wires=wires)
    qml.SingleExcitation(params[1], wires=[0, 2])
    qml.SingleExcitation(params[2], wires=[1, 3])

# Initialize the quantum device
dev = qml.device("default.qubit", wires=qubits)

# Define the cost function
@qml.qnode(dev)
def cost_fn(params):
    ansatz(params, wires=range(qubits))
    return qml.expval(H)

# Initialize parameters
params = np.random.normal(0, np.pi, 3)

# Optimize the parameters
opt = qml.GradientDescentOptimizer(stepsize=0.4)
max_iterations = 100
conv_tol = 1e-6

energy = [cost_fn(params)]
for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost_fn, params)
    energy.append(cost_fn(params))
    conv = np.abs(energy[-1] - prev_energy)
    if conv <= conv_tol:
        break

print(f"Ground state energy: {energy[-1]:.6f} Ha")

# Compare with the Hartree-Fock energy
hf_energy = qchem.hf_energy(symbols, coordinates)
print(f"Hartree-Fock energy: {hf_energy:.6f} Ha")



Ground state energy: 1.296839 Ha


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


TypeError: hf_energy() takes 1 positional argument but 2 were given